In [1]:
#映画とカクテルをマッチング（色）

In [2]:
#インポート
import pandas as pd
import sqlite3

In [3]:
#【映画】タイトル／タグ（関連性スコア）／タグ（色）
movies = pd.read_csv('C:\\Users\\t-yusuke\\Desktop\\Python_lesson\\movie_cocktail\\ml-25m\\movies.csv')
genome_scores = pd.read_csv('C:\\Users\\t-yusuke\\Desktop\\Python_lesson\\movie_cocktail\\ml-25m\\genome-scores.csv')
tag_color = pd.read_csv('C:\\Users\\t-yusuke\\Desktop\\Python_lesson\\movie_cocktail\\tag_color.csv')

#【カクテル】色別リスト
cocktail = pd.read_csv('C:\\Users\\t-yusuke\\Desktop\\Python_lesson\\movie_cocktail\\cocktail.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
db_name ='movie.db'
conn = sqlite3.connect(db_name)

In [26]:
conn

In [7]:
movies.to_sql('movies', conn, if_exists='replace')
genome_scores.to_sql('genome_scores', conn, if_exists='replace')
tag_color.to_sql('tag_color', conn, if_exists='replace')
cocktail.to_sql('cocktail', conn, if_exists='replace')

In [8]:
c = conn.cursor()

In [9]:
query = 'SELECT * FROM movies'
c.execute(query)

In [10]:
pd.merge(genome_scores, tag_color, on='tagId').head

<bound method NDFrame.head of         movieId  tagId  relevance           tag color
0             1     30    0.67625  affectionate  Pink
1             2     30    0.12675  affectionate  Pink
2             3     30    0.09550  affectionate  Pink
3             4     30    0.13150  affectionate  Pink
4             5     30    0.11875  affectionate  Pink
...         ...    ...        ...           ...   ...
594083   205072   1116    0.11675         women   Red
594084   205076   1116    0.13225         women   Red
594085   205383   1116    0.42750         women   Red
594086   205425   1116    0.10775         women   Red
594087   206499   1116    0.32850         women   Red

[594088 rows x 5 columns]>

In [11]:
scores_color = pd.merge(genome_scores, tag_color, on='tagId')

In [12]:
#出力パス
export_file_path = 'C:\\Users\\t-yusuke\\Desktop\\Python_lesson\\movie_cocktail\\output\\'

In [13]:
#Excelファイル書き出し
scores_color.to_excel(export_file_path+'scores_color.xlsx')

In [14]:
#【色別】最も関連性が高いタグ
scores_color_max = scores_color.pivot_table(index='movieId', columns='color', values='relevance', aggfunc='max').round(2)

In [81]:
#【映画のテーマカラー】最も関連性が高いタグ
scores_color_max_one = scores_color_max.idxmax(axis=1)

In [82]:
#【色別】関連性の平均
scores_color_mean = scores_color.pivot_table(index='movieId', columns='color', values='relevance', aggfunc='mean').round(2)

In [42]:
#例：ムーランルージュ（映画）とカクテルをマッチング

In [18]:
movie_title = input('好きな映画を入力してください>>')

好きな映画を入力してください>> Moulin Rouge


Moulin Rouge


In [41]:
#映画のmovieIDを確認
movies.query('title.str.contains(@movie_title)',engine='python')

,movieId,title,genres
4203,4308,Moulin Rouge (2001),Drama|Musical|Romance
8017,8731,Moulin Rouge (1952),Drama
57765,197109,Moulin Rouge (1928),Drama


In [97]:
movie_id = input('movieId　を入力してください>>')

movieId　を入力してください>> 4308


In [98]:
#映画と各色の関連性を確認
scores_color_max.query('movieId == @movie_id')

color,Black,Blue,Brown,Gray,Green,Orange,Pink,Purple,Red,White,Yellow
movieId,,,,,,,,,,,
4308,0.29,0.31,0.18,0.22,0.02,0.28,0.7,0.13,0.93,0.45,0.28


In [99]:
#最も関連性の高い色を抽出
scores_color_max_one.to_frame().query('movieId == @movie_id')

,0
movieId,
4308,Red


In [117]:
movie_id = int(movie_id)
print(movie_title + "のカラーは" + scores_color_max_one[movie_id] + "です。")

Moulin RougeのカラーはRedです。


In [118]:
#赤色のカクテルを抽出
cocktail.query('color == @cocktail_color')

,color,short,long
0,Red,"Jack Rose, Cherry Blossom","El Diablo, Campari and Soda"


In [102]:
short_long = input(shortとlong どちらがよいですか？>>')

shortとlong どちらがよいですか？>> short


In [116]:
# （予定）shortとRedを変数に変更
sql = '''
select short
from cocktail
where color == "Red"
'''

In [114]:
df_db = pd.read_sql(sql, conn)

In [77]:
df_db

,short
0,"Jack Rose, Cherry Blossom"


In [115]:
print(movie_title + "と合うカクテルは" + df_db + "です。")

                                              short
0  Moulin Rougeと合うカクテルはJack Rose, Cherry Blossomです。


In [24]:
conn.close

<function Connection.close>